In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

### Data

In [2]:
P1 = np.array([ [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0]])

P2 = np.array([ [1, 0, 0, 0],
                [0, 0, 1, 0],
                [0, -1, 0, 10]])

P3 = np.array([ [1, 1, 1, -10],
                [-1, 1, 1, 0],
                [-1, -1, 1, 10]])

P4 = np.array([ [0, 1, 1, 0],
                [0, -1, 1, 0],
                [-1, 0, 0, 10]])

y1 = np.array([0.98, 0.93])
y2 = np.array([1.01, 1.01])
y3 = np.array([0.95, 1.05])
y4 = np.array([2.04, 0.00])

In [3]:
P = np.array([P1, P2, P3, P4])
y = np.array([y1, y2, y3, y4]).reshape(4,2,1)

In [4]:
A = P[:,:2,:3]
b = P[:,:2,-1]
c = P[:, -1:, :3]
d = P[:,-1:, -1]

In [5]:
def fk(x):
    return (A@x+b)/(c@x+d)

### Function for solving Convex Feasibilty Problem for a given alpha

In [6]:
def solve(alpha):
    x = cp.Variable(3)

    const1 = [cp.norm((A[i] - y[i]@c[i])@x +  b[i]-y[i]@d[i]) <= alpha*(c[i]@x + d[i]) for i in range(4)]
    const2 = [c[i]@x + d[i] >= 0 for i in range(4)]

    constraints = const1 + const2

    prob = cp.Problem(cp.Minimize(0), constraints)
    result = prob.solve(solver=cp.ECOS)

    prob.status
    
    return x.value, prob.status

### Optimising using Bisection

In [7]:
upper = 10
lower = 0
error = 1e-4

_,res = solve(upper)

if res=='optimal':
    print(f"Start bisection search with {upper}")
else:
    print(f"Start with a bigger initial guess")

Start bisection search with 10


In [8]:
while upper-lower>error:
    alpha = (upper+lower)/2
    x,res = solve(alpha)
    if res=='optimal':
        upper = alpha
    else:
        lower = alpha

print("Bisection has been terminated")
print(f"Confidence Interval = {upper-lower}")
print(f"Estimated position = {x}")

Bisection has been terminated
Confidence Interval = 7.62939453125e-05
Estimated position = [4.91310902 5.01683617 5.19602383]
